In [1]:
from pymongo import MongoClient

In [50]:
from __future__ import print_function
import openpyxl


from collections import OrderedDict

import folium
import ggplot as gg
from ggplot import ggplot
from IPython.html.widgets import interact
import matplotlib.pyplot as plt
import mpld3
import numpy as np
import pandas as pd
from itertools import islice

%matplotlib inline
mpld3.enable_notebook()
#folium.initialize_notebook()

# axis_color = 'black'
axis_color = '#d0d0d0'

### Data processing

In [3]:
wb = openpyxl.load_workbook("data/Enerdata_Energy_Statistical_Yearbook_2016.xlsx")
wb.get_sheet_names()

['List of tables',
 'Total primary production',
 'Total balance of trade',
 'Total Energy Supply',
 'Energy Intensity of GDP',
 'Crude oil, NGL Production',
 'Crude oil, NGL Trade',
 'Crude oil, NGL Consumption',
 'Oil Products Production',
 'Oil products balance of trade',
 'Oil Products Consumption',
 'Natural gas production',
 'Natural gas balance of trade',
 'Natural Gas Consumption',
 'Coal and Lignite Production',
 'Coal and Lignite Trade',
 'Coal and Lignite Consumption',
 'Electricity production',
 'Electricity Balance of Trade',
 'Electricity Consumption',
 '  % Renewables in Electricity',
 ' % Wind Solar in Electricity',
 'CO2 Emissions',
 'CO2 Intensity',
 'Abbreviations and Glossary',
 'Geographical Coverage',
 'Sources']

In [4]:
total_pp = wb.get_sheet_by_name("Total primary production")

In [5]:
c = next(total_pp.columns)

In [6]:
for cell in c:
    print(cell.value)

Total primary production (Mtoe)
Back to list of tables
None
World
OECD
G7
BRICS
Europe
European Union
Belgium
Czech Rep.
France
Germany
Italy
Netherlands
Poland
Portugal
Romania
Spain
Sweden
United Kingdom
Norway
Turkey
CIS
Kazakhstan
Russia
Ukraine
Uzbekistan
America
North America
Canada
United States
Latin America
Argentina
Brazil
Chile
Colombia
Mexico
Venezuela
Asia
China
India
Indonesia
Japan
Malaysia
South Korea
Taiwan
Thailand
Pacific
Australia
New Zealand
Africa
Algeria
Egypt
Nigeria
South Africa
Middle-East
Iran
Kuwait
Saudi Arabia
United Arab Emirates
n.a. : non avalaible 
Source : 


In [7]:
columns = total_pp.columns
c0 = next(columns)
c1 = next(columns)
for cell in c1:
    print(cell.value)

None
None
1990
8817.292066
3444.2735780000003
2531.851674
2700.456322
1133.348496
955.0997530000001
13.035032
41.552001
111.94571300000001
190.130636
25.753112
60.560017
104.24243200000001
3.402543
40.807383
35.262654000000005
29.65907
208.006462
119.483105
25.520864000000003
1656.09751
90.977383
1296.0348040000001
126.607104
38.655288
2542.004974
1920.9426170000002
273.52946000000003
1647.413157
621.062357
48.428951999999995
104.42639
7.9288170000000004
48.181292
196.017402
144.828675
1693.23604
904.030475
281.42814500000003
168.540613
75.073133
48.82863
24.349711
10.649038000000001
26.580061
175.933953
157.418385
12.010486
676.719513
100.112749
54.861299
146.30394
114.536508
939.9515799999999
187.839802
50.378137
368.442595
110.202674
None
None


In [8]:
MONGODB_HOST = 'localhost'
MONGODB_PORT = 27017
DBS_NAME = 'world_energy'
COLLECTION_NAME = 'primary_production'

In [9]:
connection = MongoClient(MONGODB_HOST, MONGODB_PORT)
collection = connection[DBS_NAME][COLLECTION_NAME]

In [10]:
next(iter(c))

<Cell 'Total primary production'.A1>

In [11]:
c = iter(next(total_pp.columns))
for cell in c1:
    if cell.value:
        country = next(c)
        print("%s: %s" % (country.value, cell.value))
        collection.insert_one({"Country": country.value, "primary_production": cell.value})

Total primary production (Mtoe): 1990
Back to list of tables: 8817.292066
None: 3444.2735780000003
World: 2531.851674
OECD: 2700.456322
G7: 1133.348496
BRICS: 955.0997530000001
Europe: 13.035032
European Union: 41.552001
Belgium: 111.94571300000001
Czech Rep.: 190.130636
France: 25.753112
Germany: 60.560017
Italy: 104.24243200000001
Netherlands: 3.402543
Poland: 40.807383
Portugal: 35.262654000000005
Romania: 29.65907
Spain: 208.006462
Sweden: 119.483105
United Kingdom: 25.520864000000003
Norway: 1656.09751
Turkey: 90.977383
CIS: 1296.0348040000001
Kazakhstan: 126.607104
Russia: 38.655288
Ukraine: 2542.004974
Uzbekistan: 1920.9426170000002
America: 273.52946000000003
North America: 1647.413157
Canada: 621.062357
United States: 48.428951999999995
Latin America: 104.42639
Argentina: 7.9288170000000004
Brazil: 48.181292
Chile: 196.017402
Colombia: 144.828675
Mexico: 1693.23604
Venezuela: 904.030475
Asia: 281.42814500000003
China: 168.540613
India: 75.073133
Indonesia: 48.82863
Japan: 24.3

In [12]:
collection.find_one({"Country": "America"})

{'Country': 'America',
 '_id': ObjectId('58ec628a421aa9295e5eb15e'),
 'primary_production': 273.52946000000003}

In [ ]:
data = total_pp.values
data = list(data)
idx = [r[0] for r in data]
data = (islice(r, 1, None) for r in data)
df = pd.DataFrame(data, index=idx)

In [76]:
countries_df = df.ix[countries_names].dropna(how="any")
countries_df

In [91]:
countries_df["names"] = countries_df.index

In [99]:
countries_df.head(10)

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,names
United Arab Emirates,110.202674,127.291886,133.966319,131.291312,133.570659,137.036872,142.069845,147.200984,150.940993,142.755072,...,167.763148,174.173584,187.930461,194.359399,201.677469,204.156991,215.326453,5.47102,2.15395,United Arab Emirates
Argentina,48.428952,50.228744,54.334610,57.140763,62.438018,67.138228,72.589379,76.804605,79.548296,80.516173,...,81.078867,79.521730,77.265024,75.219700,71.439991,71.813378,72.850213,1.44379,-0.858411,Argentina
Australia,157.418385,166.896825,171.473606,174.265288,172.876399,186.896552,189.761513,201.124638,216.590003,213.682421,...,295.341646,310.744559,299.603856,315.778757,343.914031,365.297671,354.636928,-2.91837,2.81953,Australia
Belgium,13.035032,12.921057,12.679260,12.008510,11.589174,11.870380,12.319165,13.344168,13.040352,13.851624,...,14.904178,15.553498,16.143168,14.207204,14.911907,12.753738,10.856239,-14.878,-1.52459,Belgium
Brazil,104.426390,105.017217,104.991470,105.907911,110.959911,112.323752,118.883402,126.602118,133.733805,141.628468,...,231.286815,247.331549,250.028624,252.746297,253.755161,267.167061,280.186009,4.87296,4.35539,Brazil
Canada,273.529460,284.913468,294.132974,315.956638,338.580200,349.020647,358.518094,365.281761,367.777288,366.852301,...,388.792522,394.788525,407.199579,416.394630,434.176089,456.311311,456.365885,0.0119598,1.32224,Canada
Chile,7.928817,8.349638,8.631508,8.231570,8.317267,8.303658,8.330793,8.364261,8.108323,7.959366,...,10.206554,9.220187,9.878568,13.028320,14.981594,15.663605,13.943405,-10.9821,3.2886,Chile
China,904.030475,909.944385,926.381856,958.123370,1014.517574,1064.606905,1091.030457,1079.780620,1080.283115,1075.474028,...,2112.809006,2258.883283,2469.656283,2491.143730,2567.410466,2623.894703,2640.316608,0.62586,5.74869,China
Colombia,48.181292,46.351154,47.877525,48.557422,50.425206,59.243699,64.836673,66.729941,72.707093,75.551020,...,98.211010,105.935129,120.367560,124.531820,125.555413,126.271790,124.619231,-1.30873,3.69323,Colombia
Czech Rep.,41.552001,39.186132,36.288196,35.754565,33.304349,32.448253,33.122901,33.443971,31.496143,28.668809,...,31.245498,31.624393,32.172386,32.620006,29.998991,28.627472,27.276806,-4.71808,-0.76146,Czech Rep.


### Maps

In [19]:
map_osm = folium.Map(location=[45.5236, -122.6750])

In [20]:
map_osm

In [21]:
map = folium.Map(location=[48, -102], zoom_start=2)

In [94]:
map.choropleth(geo_path=r'geojson/countries.json', data=countries_df, columns=["names", 1], 
               key_on='feature.properties.name',
              fill_color='YlGn', fill_opacity=0.7, line_opacity=0.2)

In [95]:
map

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [40]:
import json
countries_geojson = json.load(open("geojson/countries.json"))

In [41]:
countries_names = [feature["properties"]["name"] for feature in countries_geojson["features"]]
countries_names

['Afghanistan',
 'Angola',
 'Albania',
 'United Arab Emirates',
 'Argentina',
 'Armenia',
 'Antarctica',
 'Fr. S. Antarctic Lands',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Burundi',
 'Belgium',
 'Benin',
 'Burkina Faso',
 'Bangladesh',
 'Bulgaria',
 'Bahamas',
 'Bosnia and Herz.',
 'Belarus',
 'Belize',
 'Bolivia',
 'Brazil',
 'Brunei',
 'Bhutan',
 'Botswana',
 'Central African Rep.',
 'Canada',
 'Switzerland',
 'Chile',
 'China',
 "Côte d'Ivoire",
 'Cameroon',
 'Dem. Rep. Congo',
 'Congo',
 'Colombia',
 'Costa Rica',
 'Cuba',
 'N. Cyprus',
 'Cyprus',
 'Czech Rep.',
 'Germany',
 'Djibouti',
 'Denmark',
 'Dominican Rep.',
 'Algeria',
 'Ecuador',
 'Egypt',
 'Eritrea',
 'Spain',
 'Estonia',
 'Ethiopia',
 'Finland',
 'Fiji',
 'Falkland Is.',
 'France',
 'Gabon',
 'United Kingdom',
 'Georgia',
 'Ghana',
 'Guinea',
 'Gambia',
 'Guinea-Bissau',
 'Eq. Guinea',
 'Greece',
 'Greenland',
 'Guatemala',
 'Guyana',
 'Honduras',
 'Croatia',
 'Haiti',
 'Hungary',
 'Indonesia',
 'India',
 'Ireland',

In [43]:
for cell in c0:
    if cell.value in countries_names:
        print(cell.value)

Belgium
Czech Rep.
France
Germany
Italy
Netherlands
Poland
Portugal
Romania
Spain
Sweden
United Kingdom
Norway
Turkey
Kazakhstan
Russia
Ukraine
Uzbekistan
Canada
United States
Argentina
Brazil
Chile
Colombia
Mexico
Venezuela
China
India
Indonesia
Japan
Malaysia
Taiwan
Thailand
Australia
New Zealand
Algeria
Egypt
Nigeria
South Africa
Iran
Kuwait
Saudi Arabia
United Arab Emirates
